# The interesting topics in this year

In [1]:
from semanticscholar import SemanticScholar
import time
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

def get_paper_citation(title):
    sch = SemanticScholar()
    time.sleep(1)
    title = title.replace('-', ' ')
    try:
        papers = sch.search_paper(title)
        if len(papers) > 0:
            return papers[0].citationCount
    except:
        print(f'Error in get citation for {title}')
    
    return 0

In [2]:
def get_conf_topics(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal']):
    df_conf = pd.read_csv(conf_path)

    save_path = conf_path + '_emerging.xlsx'
    writer = pd.ExcelWriter(save_path, engine="xlsxwriter")

    # if topics within the list of topics
    df_conf_topics = df_conf[df_conf['topic'].str.contains('|'.join(topics), case=False, na=False)].copy()

    ss_citations = []
    for title in tqdm(df_conf_topics['title'], ncols=80):
        ss_citations.append(get_paper_citation(title))
        time.sleep(1)
    
    df_conf_topics['ss_citations'] = ss_citations
    df_conf_topics['google_scholar_link'] = df_conf_topics['title'].apply(lambda x: 'https://scholar.google.com/scholar?q=' + x.replace(' ', '+'))

    df_conf_topics[['title','year','source','ss_citations','google_scholar_link', 'topic']].to_excel(writer, sheet_name='topics', index=False)
    writer.close()
    print(f'Saved to {save_path}')

# 2023

## 2023, ICLR, May

In [3]:
# get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal', 'diffusion models', 'llm', 'self-supervision'])

## 2023, CVPR, June

In [4]:
# get_conf_topics_all(conf_path='data/2023/6_cvpr.csv_topics.csv', topics=['3d', 'zero_few-shot', 'transformer', 'segmentation', 'multimodal', 
# 'imitation learning', 'radiance field', 'motion', 'diffusion models', 'multi-agent'])

## 2023, ICML, July

In [5]:
# get_conf_topics_all(conf_path='data/2023/7_icml.csv_topics.csv', topics=['zero_few-shot', 'graph', 'optimization', 'reinforcement learning', 'representation',
# 'motion', 'decision trees', 'radiance field', 'diffusion models', 'federated learning'])

## 2023, ICCV, Oct.

In [6]:
# get_conf_topics(conf_path='data/2023/10_ICCV.csv_topics.csv', topics=['object detection', '3d', 'segmentation', 'transformer', 'zero_few-shot'])

## 2023, NIPS, Dec.

In [7]:
# get_conf_topics(conf_path='data/2023/12_nips.csv_topics.csv', topics=['object detection', 'zero_few-shot', 'graph', 'optimization', 'llm', 'radiance field', 'motion', 'differential equation solving', 'rl'])

# 2024

## 2024, ICLR, May

In [8]:
# get_conf_topics(conf_path='data/2024/5_iclr.csv_topics.csv', topics=['graph', 'zero_few-shot', 'llm', 'transformer', 'generative model', 'reward model', 'watermark', 'foundation models', 'model merging', 'diffusion models'])

## 2024, CVPR, June

In [9]:
# get_conf_topics(conf_path='data/2024/6_cvpr.csv_topics.csv', topics=['3d', 'generative model', 'segmentation', 'multimodal', 'zero_few-shot', 'watermark', 'streaming', 'token', 'autonomous driving', 'entangle'])

## 2024, ICML, July

In [10]:
# get_conf_topics(conf_path='data/2024/7_icml.csv_topics.csv', topics=['deformation', 'autonomous driving', 'object detection', 'splatting', 'token', 'graph', 'zero_few-shot', 'llm', 'optimization', 'transformer'])

## 2024, NIPS, Dec.

In [11]:
# get_conf_topics(conf_path='data/2024/12_nips.csv_topics.csv', topics=['zero_few-shot', 'graph', 'llm', 'optimization', 'transformer', '4D', 'splatting', 'diffusion transformer', 'state space model', 'text-to-video'])

# 2025

## 2025, ICLR

In [12]:
# get_conf_topics(conf_path='data/2025/5_iclr.csv_topics.csv', topics=['graph', 'zero_few-shot', 'llm', 'generative model', 'transformer', 'Diffusion models', 'Flow matching', 'Flow', 'Agentic AI', 'Mathematical reasoning', 'splatting', 'diffusion transformer', 'state space model', 'Flow matching', 'RAG' ])

In [13]:
get_conf_topics(conf_path='data/2025/6_cvpr.csv_topics.csv', topics=['3d', '3d vision', 'video', 'generative model', 'multimodal', 'Flow matching', 'direct preference', 'Membership Inference', 'tuning-free', 'EEG'])

 11%|███▊                               | 134/1230 [1:24:45<25:21:23, 83.29s/it]

Error in get citation for Critic V: VLM Critics Help Catch VLM Errors in Multimodal Reasoning


 23%|███████▉                           | 281/1230 [3:10:35<16:32:15, 62.73s/it]

Error in get citation for Uncertainty Meets Diversity: A Comprehensive Active Learning Framework for Indoor 3D Object Detection


 23%|████████▏                          | 287/1230 [3:20:36<20:54:46, 79.84s/it]

...

Error in get citation for Generating Multimodal Driving Scenes via Next Scene Prediction


 70%|█████████████████████████▎          | 865/1230 [9:14:16<3:23:00, 33.37s/it]

Error in get citation for S2D LFE: Sparse to Dense Light Field Event Generation


100%|████████████████████████████████████| 1230/1230 [11:20:46<00:00, 33.21s/it]

Saved to data/2025/6_cvpr.csv_topics.csv_emerging.xlsx


In [14]:
get_conf_topics(conf_path='data/2025/7_icml.csv_topics.csv', topics=['graph', 'zero_few-shot', 'llm', 'optimization', 'transformer', 'contamination', 'direct preference', 'remote sensing', 'tuning-free', 'video'])



  1%|▎                                      | 21/2854 [01:35<2:12:43,  2.81s/it]

Error in get citation for Not All Wrong is Bad: Using Adversarial Examples for Unlearning


  2%|▋                                      | 51/2854 [06:11<7:34:02,  9.72s/it]

Error in get citation for Reducing Variance of Stochastic Optimization for Approximating Nash Equilibria in Normal Form Games


  2%|▉                                      | 65/2854 [08:24<9:50:41, 12.71s/it]

...

Error in get citation for World Model Implanting for Test time Adaptation of Embodied Agents


 99%|███████████████████████████████████▊| 2839/2854 [18:27:20<04:11, 16.74s/it]

Error in get citation for Revisiting Neural Networks for Few Shot Learning: A Zero Cost NAS Perspective


100%|████████████████████████████████████| 2854/2854 [18:41:02<00:00, 23.57s/it]


Saved to data/2025/7_icml.csv_topics.csv_emerging.xlsx
